[![Open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/dkedar7/fast_dash/blob/docs/docs/Examples/06_choropleth_maps.ipynb)

This notebook is optimized to run in Google Colab.

In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

from urllib.request import urlopen
import json

from fast_dash import fastdash, Graph

In [3]:
# Counties sample data
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

counties_df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv", dtype={"fips": str})

# US Ag exports sample data
us_ag_exports_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

for col in us_ag_exports_df.columns:
    us_ag_exports_df[col] = us_ag_exports_df[col].astype(str)

us_ag_exports_df['text'] = us_ag_exports_df['state'] + '<br>' + \
    'Beef ' + us_ag_exports_df['beef'] + ' Dairy ' + us_ag_exports_df['dairy'] + '<br>' + \
    'Fruits ' + us_ag_exports_df['total fruits'] + ' Veggies ' + us_ag_exports_df['total veggies'] + '<br>' + \
    'Wheat ' + us_ag_exports_df['wheat'] + ' Corn ' + us_ag_exports_df['corn']

In [4]:
@fastdash(theme="flatly")
def visualize_states_and_counties(a: int) -> (Graph, Graph):
    """
    Fast Dash allows quick and easy visualization of various geographies.\
    This application plots states and counties using two differnent basemaps.\
    Reference: https://plotly.com/python/mapbox-county-choropleth/.
    """

    # Plotly-native graph map
    fig = go.Figure(data=go.Choropleth(
        locations=us_ag_exports_df['code'],
        z=us_ag_exports_df['total exports'].astype(float),
        locationmode='USA-states',
        colorscale='Reds',
        autocolorscale=False,
        text=us_ag_exports_df['text'], # hover text
        marker_line_color='white', # line markers between states
        colorbar_title="Millions USD"
    ))

    fig.update_layout(
        title_text='2011 US Agriculture Exports by State<br>(Hover for breakdown)',
        geo = dict(
            scope='usa',
            projection=go.layout.geo.Projection(type = 'albers usa'),
            showlakes=True, # lakes
            lakecolor='rgb(255, 255, 255)'),
    )
   
    plotly_express = fig

    # Mapbox graph map
    fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=counties_df.fips, z=counties_df.unemp,
                                        colorscale="Viridis", zmin=0, zmax=12, marker_line_width=0))
    fig.update_layout(mapbox_style="light", mapbox_accesstoken="...",
                    mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    mapbox = fig
    return plotly_express, mapbox